In [1]:
import torch
from torch import nn, optim, autograd
import torchvision 
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from dataclasses import dataclass
import time
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
%matplotlib inline
torch.set_num_threads(1)
torch.manual_seed(1)

In [2]:
@dataclass
class Hyperparameter:
    num_classes: int        = 1
    batchsize: int          = 128
    num_epochs: int         = 20
    latent_size: int        = 32 #used in generatinf random noise?
    n_critic: int           = 5
    critic_size: int        = 3
    generator_size: int     = 2
    critic_hidden_size: int = 1024
    gp_lambda: float        = 10. #used in gradient
        
hp = Hyperparameter()

In [3]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(2, 256),# 2-input generator (noise, energy: en_p)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1), #1 output distance
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 1, 1)


In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(1, 100)
        
        self.model = nn.Sequential(
            nn.Linear(3, 1024), # 3-input discriminator (real data = d_p,label=,en_p,generated data = distance)
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid() #1-output discriminator (real vs fake/generated)
        )
    
    def forward(self, x, labels):
        x = x.view(128)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()



In [5]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize Generator and discriminator
generator = Generator()
discriminator = Discriminator()

if torch.cuda.is_available():
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    
discriminator_optimizer = optim.AdamW(discriminator.parameters(), lr=1e-4,betas=(0., 0.9))
generator_optimizer = optim.AdamW(generator.parameters(), lr=1e-4,betas=(0., 0.9))

In [6]:
water_df = pd.read_pickle("water_dataset.pkl")

In [14]:
x = water_df["en_p"].values.reshape(-1,1)
#dependent variable distance
y = water_df["dist_p"].values.reshape(-1,1)

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [19]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

device = get_device()
device = get_device()
x_train_torch = torch.from_numpy(x_train).float().to(device)
y_train_torch = torch.from_numpy(y_train).float().to(device)
trainset = torch.utils.data.TensorDataset(x_train_torch, y_train_torch)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, drop_last=True)

print('the data is ok')

<built-in method count of tuple object at 0x000001F9456FDC40>
the data is ok


In [9]:

img_list, generator_losses, discriminator_losses = [], [], []
iters = 0
all_labels = torch.eye(hp.num_classes, dtype=torch.float32, device="cuda")
fixed_noise = torch.randn((80, hp.latent_size), device="cuda")
fixed_class_labels = all_labels[[i for i in list(range(hp.num_classes)) for idx in range(8)]]
grad_tensor = torch.ones((hp.batchsize, 1), device="cuda")

start_time = time.time()
for epoch in range(hp.num_epochs):
    for batch_idx, data in enumerate(trainloader, 0):
        real_images, real_class_labels = data[0].to("cuda"), all_labels[data[1]].to("cuda")
        
        # Update critic
        discriminator_optimizer.zero_grad()
        
        discriminator_output_real = discriminator(real_images, real_class_labels)
        discriminator_loss_real = discriminator_output_real.mean()

        noise = torch.randn((hp.batchsize, hp.latent_size), device="cuda")
        with torch.no_grad(): fake_image = generator(noise, real_class_labels)
        discriminator_output_fake = discriminator(fake_image, real_class_labels)
        discriminator_loss_fake = discriminator_output_fake.mean()

        alpha = torch.rand((hp.batchsize, 1), device="cuda")
        interpolates = (alpha.view(-1, 1, 1, 1) * real_images + ((1. - alpha.view(-1, 1, 1, 1)) * fake_image)).requires_grad_(True)
        d_interpolates = discriminator(interpolates, real_class_labels)
        gradients = autograd.grad(d_interpolates, interpolates, grad_tensor, create_graph=True, only_inputs=True)[0]
        gradient_penalty = hp.gp_lambda * ((gradients.view(hp.batchsize, -1).norm(dim=1) - 1.) ** 2).mean()

        discriminator_loss = -discriminator_loss_real + discriminator_loss_fake  + gradient_penalty
        
        discriminator_loss.backward()
        discriminator_optimizer.step()

        if batch_idx % hp.n_critic == 0:
            # Update Generator
            generator_optimizer.zero_grad()
            
            fake_class_labels = all_labels[torch.randint(hp.num_classes, size=[hp.batchsize])]
            noise = torch.randn((hp.batchsize, hp.latent_size), device="cuda")
            fake_image = generator(noise, fake_class_labels)
            discriminator_output_fake = discriminator(fake_image, fake_class_labels)
            generator_loss = -discriminator_output_fake.mean()
            
            generator_loss.backward()
            generator_optimizer.step()
        
        # Output training stats
        if batch_idx % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"[{epoch:>2}/{hp.num_epochs}][{iters:>7}][{elapsed_time:8.2f}s]\t"
                  f"d_loss/g_loss: {discriminator_loss.item():4.2}/{generator_loss.item():4.2}\t")
       
        # Save Losses for plotting later
        generator_losses.append(generator_loss.item())
        discriminator_losses.append(discriminator_loss.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == hp.num_epochs - 1) and (batch_idx == len(trainloader) - 1)):
            with torch.no_grad(): fake_images = generator(fixed_noise, fixed_class_labels).cpu()
            img_list.append(vutils.make_grid(fake_images, padding=2, normalize=True))
            
        iters += 1

IndexError: tensors used as indices must be long, byte or bool tensors